# Separate a collapsed column into multiple rows

Sometimes a variable has multiple elements stored in one row, where they are each separated by a delimeter (i.e. `,`, `\t`, `\`, etc.). Often it more useful to separate those elements into their own records so our [data becomes tidy](http://vita.had.co.nz/papers/tidy-data.pdf).

## Goal

If a variable contains observations with multiple delimited values, this notebook will show you how to separate the values and place each one in its own row as a pandas DataFrame.

### Stack Overflow

This would not be possible without [this answer from Stack Overflow](https://stackoverflow.com/a/28182629/7954106) regarding the separation of elements in a column into multiple rows.

### Load necessary modules

In [1]:
import pandas as pd

### Load necessary data

Today we'll be using [Chicago Public Schools (CPS) School Year 2018-2019 school profile data](https://cenuno.github.io/pointdexter/reference/cps_sy1819.html).

In [2]:
relevant_columns = ["school_id", "short_name", 
                    "primary_category", "grades_offered_all"]

cps_sy1819 = pd.read_csv("../raw_data/cps_sy1819_profiles.csv")[relevant_columns]
cps_sy1819.head()

,school_id,short_name,primary_category,grades_offered_all
0,609760,CARVER MILITARY HS,HS,"9,10,11,12"
1,609780,MARINE LEADERSHIP AT AMES HS,HS,"7,8,9,10,11,12"
2,610304,PHOENIX MILITARY HS,HS,"9,10,11,12"
3,610513,AIR FORCE HS,HS,"9,10,11,12"
4,610390,RICKOVER MILITARY HS,HS,"9,10,11,12"


### [Cast each element in the `Series` as a string](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) and [split each string by the delimiter](https://docs.python.org/3.7/library/stdtypes.html#str.split). 

Afterwards, store the `Series` object as a `list` of lists.

In [3]:
cps_sy1819["grades_offered_all"].str.split(",")[0:5]

0          [9, 10, 11, 12]
1    [7, 8, 9, 10, 11, 12]
2          [9, 10, 11, 12]
3          [9, 10, 11, 12]
4          [9, 10, 11, 12]
Name: grades_offered_all, dtype: object

### Store results in a [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html#pandas.DataFrame)

After coverting the `list` to a `DataFrame`, you'll notice a lot of `None` values. Behind the hood, `pandas` is ensuring each record has 13 columns. Since not all schools serve 13 grades, they'll get `None` values.


In [4]:
pd.DataFrame(cps_sy1819["grades_offered_all"].str.split(",").tolist(),
             index=cps_sy1819.index).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,9,10,11,12,None,None,None,None,None,None,None,None,None
1,7,8,9,10,11,12,None,None,None,None,None,None,None
2,9,10,11,12,None,None,None,None,None,None,None,None,None
3,9,10,11,12,None,None,None,None,None,None,None,None,None
4,9,10,11,12,None,None,None,None,None,None,None,None,None


### Covert the `DataFrame` to a `Series` with a multi-level index via [`stack()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html)

> The function is named by analogy with a collection of books being re-organised from being side by side on a horizontal position (the columns of the dataframe) to being stacked vertically on top of of each other (in the index of the dataframe).

![Stripes book](../visuals/stripes_book.gif)

In [5]:
pd.DataFrame(cps_sy1819["grades_offered_all"].str.split(",").tolist(),
             index=cps_sy1819.index).head().stack()[0:10]

0  0     9
   1    10
   2    11
   3    12
1  0     7
   1     8
   2     9
   3    10
   4    11
   5    12
dtype: object

### Putting it all together

[Reset the index](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) to add the old index is added as a column

We will no longer need the lower level index for each grade in each school so it can be dropped. Instead, the higher level index for each school is what we will use moving forward.

*Note: I renamed the columns to make them easier to understand.*

In [6]:
grades_df = pd.DataFrame(cps_sy1819["grades_offered_all"].str.split(",").tolist(),
             index=cps_sy1819.index).stack()

grades_df = grades_df.reset_index()

grades_df = grades_df.drop("level_1", axis=1)

grades_df.columns = ["school_index", "grade_offered"]

grades_df.head(10)

,school_index,grade_offered
0,0,9
1,0,10
2,0,11
3,0,12
4,1,7
5,1,8
6,1,9
7,1,10
8,1,11
9,1,12


### Merge `grades_df` onto `cps_sy1819` via a left join

We'll use the indices from `cps_sy1819` on the left-hand side, along with the `school_index` values from `grades_df`, to perfom the join.

After the join, we'll drop the `school_index` feature since it's redundant information in that `school_id` already indicates that each record is related to one particular CPS school.

In [7]:
cps_sy1819 = pd.merge(cps_sy1819,
                      grades_df,
                      left_index=True,
                      right_on="school_index",
                      how="left").drop("school_index", axis=1)

cps_sy1819.head(10)

,school_id,short_name,primary_category,grades_offered_all,grade_offered
0,609760,CARVER MILITARY HS,HS,"9,10,11,12",9
1,609760,CARVER MILITARY HS,HS,"9,10,11,12",10
2,609760,CARVER MILITARY HS,HS,"9,10,11,12",11
3,609760,CARVER MILITARY HS,HS,"9,10,11,12",12
4,609780,MARINE LEADERSHIP AT AMES HS,HS,"7,8,9,10,11,12",7
5,609780,MARINE LEADERSHIP AT AMES HS,HS,"7,8,9,10,11,12",8
6,609780,MARINE LEADERSHIP AT AMES HS,HS,"7,8,9,10,11,12",9
7,609780,MARINE LEADERSHIP AT AMES HS,HS,"7,8,9,10,11,12",10
8,609780,MARINE LEADERSHIP AT AMES HS,HS,"7,8,9,10,11,12",11
9,609780,MARINE LEADERSHIP AT AMES HS,HS,"7,8,9,10,11,12",12
